In [150]:
import numpy as np
from itertools import product
import functools
from qiskit.quantum_info import Statevector, Pauli, SparsePauliOp, partial_trace
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from scipy.sparse.linalg import expm_multiply, minres, eigsh, expm, eigsh
from scipy.linalg import ishermitian
from hamiltonian_learning import reconstruct_hamiltonian,hamiltonian_to_vector, create_hamiltonian_lattice, create_constraint_matrix, create_klocal_pauli_basis,simple_purify_hamiltonian, sample_pauli_basis

Create the Hamiltonian that we want to find the Gibbs State of:

In [151]:
N=7
k = 2
hamiltonian = create_hamiltonian_lattice(N, 0.1, -1.0)
#We can add some noise to the hamiltonian
noisy_hamiltonian = (hamiltonian + 1e-5 * create_hamiltonian_lattice(N, -1, 3)).simplify()

We could try to implement a thermometer.

In [152]:
# thermometer = SparsePauliOp.from_list([("I"*N + "Z",1.0)])    
# hamiltonian= (hamiltonian^"I") + thermometer
# N += 1

Now we can create the exact thermal state of the hamiltonian, if we want to try the faulty implementation, the error is modeled by adding noise to our hamiltonian.

In [153]:
thermal_state = simple_purify_hamiltonian(hamiltonian, noise=0)
# thermal_state = simple_purify_hamiltonian(noisy_hamiltonian, noise=0)

We now choose our set of constraints and sample over the needed basis to construct all the commutators [A,S] we need.

In [154]:

sampling_basis = create_klocal_pauli_basis(N, 2 * k)
epsilon = 1e-8
sampled_pauli = sample_pauli_basis(thermal_state, sampling_basis, 1000, noise=epsilon)
Aq_basis = list(create_klocal_pauli_basis(N, k + 1))
Sm_basis = list(create_klocal_pauli_basis(N, k))
constraint_matrix = create_constraint_matrix(
    sampled_pauli, Aq_basis=Aq_basis, Sm_basis=Sm_basis
)
print("The shape of the constraint matrix is:",constraint_matrix.shape)


The shape of the constraint matrix is: (255, 75)


Now we need to find the singular values and singular vectors of K.

In [155]:
KTK = constraint_matrix.T.conj().dot(constraint_matrix)
eigvals, eigvecs = np.linalg.eigh(KTK.todense())
print("The first singular values of K are")
print(eigvals[:4])


The first singular values of K are
[2.82016082e-15 1.23666843e-04 2.21730670e-04 3.44749616e-04]


Finally, we can compare the error between the actual hamiltonian that has been implemented and the one we learned.

In [157]:
original_vector = hamiltonian_to_vector(Sm_basis, hamiltonian)
noisy_hamiltonian_vector = hamiltonian_to_vector(Sm_basis, noisy_hamiltonian)
rec_hams = [reconstruct_hamiltonian(Sm_basis, eigvecs[:, i]) for i in range(5)]
rec_vectors = [hamiltonian_to_vector(Sm_basis, rec_hams[i]) for i in range(5)]
rec_vectors = [r / np.linalg.norm(r) for r in rec_vectors]
print(f"The actual error between normalized hamiltonians is: {np.linalg.norm( rec_vectors[0] *np.linalg.norm(original_vector) - original_vector  ) :.2e}" )
print(f"The distance between the original and the noisy hamiltonian is: { np.linalg.norm( original_vector - noisy_hamiltonian_vector) :.2e}" )
# print(f"The expected error was less than:{ np.linalg.norm(original_vector) * epsilon *np.sqrt(1/eigvals[1:].sum()):.2e}" )

The actual error between normalized hamiltonians is: 1.79e-06
The distance between the original and the noisy hamiltonian is: 8.31e-05
